In [41]:
import os

In [42]:
from Utilities.email_sending import send_plain_email, send_html_email

EMAIL_PASSWORD = os.getenv('EMAIL_PASSWORD') # the password needs to be set as EMAIL_PASSWORD in system variables of the computer where the process is running, it is located in S:/Algo/email_password.txt
if EMAIL_PASSWORD is None:
    raise ValueError("EMAIL_PASSWORD system variable not set")

RECIPIENT = ["zubal_andrej@energytrading.sk", "azubal@seznam.cz"] # can also be a list of recipients

In [43]:
def convert_to_html_content(input_string):
    # Split the input string by the newline character
    lines = input_string.split('\n')
    # Wrap each line in a <p> tag
    html_lines = [f"<p>{line}</p>" for line in lines]
    # Join the lines into a single HTML string
    html_content = '\n'.join(html_lines)
    return html_content


def convert_dataframe_to_html_content(df):
    # Convert the DataFrame to an HTML table
    html_content = '<table border="1" cellpadding="5" cellspacing="0" style="border-collapse: collapse;">\n'

    # Add the header row
    name = 'Order Book'
    html_content += '  <tr>\n'
    html_content += f'    <th>{name}</th>\n'
    for column in df.columns:
        html_content += f'    <th>{column}</th>\n'
    html_content += '  </tr>\n'

    # Add the data rows
    for idx, row in df.iterrows():
        html_content += '  <tr>\n'
        html_content += f'    <td>{idx}</td>\n'  # Add the index as the first column
        for item in row:
            html_content += f'    <td>{item}</td>\n'
        html_content += '  </tr>\n'

    # Close the HTML table
    html_content += '</table>'

    return html_content

In [44]:
import cfRestApiV3 as cfApi
import datetime
import pandas as pd
import json
import matplotlib.pyplot as plt

In [45]:
# accessible on your Account page under Settings -> API Keys
apiPublicKey = "X0J+mG76qUHLfr7G4Yk0/cSX7+T7cApznHo2opaEOOqNZKed2AmDFuMI"
# accessible on your Account page under Settings -> API Keys
apiPrivateKey = "YMyZG6GhXNmvW/ZvLzFp5zxTX+dmfPJCp8Vzamz8LJ+CBmQmloF7yBC3/sD3mNrMw8i8CtNqMnd9233v3+6kpUlT"

In [46]:
# use "api.cryptofacilities.com" if your IP is whitelisted (Settings -> API Keys -> IP Whitelist)
apiPath = "https://www.cryptofacilities.com"
timeout = 20
checkCertificate = True  # when using the test environment, this must be set to "False"
useNonce = False  # nonce is optional

cfPublic = cfApi.cfApiMethods(
    apiPath, timeout=timeout, checkCertificate=checkCertificate)
cfPrivate = cfApi.cfApiMethods(apiPath, timeout=timeout, apiPublicKey=apiPublicKey,
                               apiPrivateKey=apiPrivateKey, checkCertificate=checkCertificate, useNonce=useNonce)

In [47]:
from datetime import datetime

initial_value_usd=866.57
initial_date=datetime.strptime("2024-12-04", "%Y-%m-%d").date()
today_date = datetime.today().date()

# Calculate the difference in days
days_difference = (today_date - initial_date).days
days_difference

1

In [48]:
%store -r yesterdays_value_usd

In [49]:
todays_value_usd=json.loads(cfPrivate.get_accounts())['accounts']['flex']['currencies']['USD']['value']

In [50]:
todays_value_usd

886.19533310485

In [51]:
profit_24h=todays_value_usd-yesterdays_value_usd
interest_rate_24h=(1+profit_24h/yesterdays_value_usd)**365.0
profit_overall=todays_value_usd-initial_value_usd
avg_daily_profit=profit_overall/days_difference
interest_rate_overall=(1+profit_overall/initial_value_usd)**(365/days_difference)

In [52]:
html_content = f"""\
<html>
    <body>
        <h1>The job was successful </h1>
        <p>profit_24h: {profit_24h}</p>
        <p>interest_rate_24h: {interest_rate_24h}</p>

        <p>profit_overall: {profit_overall}</p>
        <p>interest_rate_overall: {interest_rate_overall}</p>

        <p>avg_daily_profit: {avg_daily_profit}</p>

    </body>
</html>
"""

send_html_email(
    RECIPIENT, 
    "AUTOMATIC JOBS - SUCCESS daily_profit_increment", "", html_content, "",
        email_password=EMAIL_PASSWORD

        )